# Create YOLO Dataset

In this notbook the YOLO dataset is created containing both the player and the ball annotations

### imports

In [1]:
import os
import json
import shutil
import cv2

In [ ]:
#read annotation json
with open('./00_Dataset/annotations_player.json') as f:
    data = json.load(f)

## create the folder structure

In [3]:
path = 'datasets/YOLOPlayerBall'

# create folder structure for YOLO Dataset
os.makedirs(path, exist_ok=True)
#subfolders for images and labels
os.makedirs(path+'/images', exist_ok=True)
os.makedirs(path+'/labels', exist_ok=True)
#train, test and validation subfolders
os.makedirs(path+'/images/train', exist_ok=True)
os.makedirs(path+'/images/validation', exist_ok=True)
os.makedirs(path+'/images/test', exist_ok=True)
os.makedirs(path+'/labels/train', exist_ok=True)
os.makedirs(path+'/labels/validation', exist_ok=True)
os.makedirs(path+'/labels/test', exist_ok=True)

In [4]:
# create the yaml file
with open(path+'/annotations.yaml', 'w') as f:
    f.write("train: ../images/train\n")
    f.write("val: ../images/validation\n")
    f.write("test: ../images/test\n")
    f.write("nc: 2\n")
    f.write("names: \n")
    f.write("   0: ball\n")
    f.write("   1: person\n")

In [ ]:
def convert_player_to_yolo_bb(player, img_width, img_height):
        # get the center point
        x = (player['xtl'] + player['xbr']) / 2
        y = (player['ytl'] + player['ybr']) / 2

        # normalize the center point
        x /= img_width
        y /= img_height
        x = round(x, 6)
        y = round(y, 6)

        # normalize the width and height    
        w = (player['xbr'] - player['xtl']) / img_width
        h = (player['ybr'] - player['ytl']) / img_height
        w = round(w, 6)
        h = round(h, 6)

        return 1, x, y, w, h


def convert_point_to_yolo_bb(center_point, img_width, img_height):
    # normalize the center point
    x, y = center_point
    x /= img_width
    y /= img_height
    x = round(x, 6)
    y = round(y, 6)

    # size for 20x20px at a resolution of 1280x720
    size_x = 0.015625
    size_y = 0.027776

    return 0, x, y, size_x, size_y

def create_dataset():
    yolo_dataset_dir = path+"/images"

    # loop over each frame in the json file
    for subset in data['subsets']:
        subset_name = subset['name']
        resolution = subset['resolution']
        for video in subset['videos']:
            video_name = video['name']
            
            for clip in video['clips']:
                clip_name = clip['name']
                for frame_num, frame in clip['frames_with_objects'].items():

                    #get split
                    split = frame["split"]
                    
                    # get the image path
                    img_path = os.path.join(
                        "./00_Dataset",
                        subset_name,
                        video_name,
                        clip_name,
                        str(int(frame_num)) + ".jpg"
                    )

                    #create new filename
                    file_name = subset_name + "_" + video_name + "_" + clip_name + "_" + str(int(frame_num)) 
                    dest_path = os.path.join(yolo_dataset_dir, split, file_name+ ".jpg")

                   # Copy the image to the YOLO dataset directory
                    try:
                        shutil.copy2(img_path, dest_path)
                    except FileNotFoundError as e:
                        print(e)
                    except Exception as e:
                        print(f"Error copying file {img_path}: {e}")

                    # create the label file
                    with open(f"{path}/labels/{split}/{file_name}.txt", 'w') as f:
                        for player in frame['players']:
                                # get the bounding box
                                c, x, y, w, h = convert_player_to_yolo_bb(player, resolution[0], resolution[1])

                                # write the label to the file
                                f.write(f"{c} {x} {y} {w} {h}\n")   

                        for ball in frame['balls']:
                            if ball['visibility'] not in ['Outside'] and ball['trajectory'] not in ['', 'Static']:
                                point = (ball['x'], ball['y'])
                                # get the bounding box
                                c, x, y, w, h = convert_point_to_yolo_bb(point, resolution[0], resolution[1])

                                # write the label to the file
                                f.write(f"{c} {x} {y} {w} {h}\n")                   

In [6]:
create_dataset()